![Kayak](https://seekvectorlogo.com/wp-content/uploads/2018/01/kayak-vector-logo.png)

# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

# Library imports

In [3]:
import requests
import re
import pandas as pd
import plotly.express as px
import json
import plotly.io as pio
import plotly.graph_objects as go

## City data API Requests

In [4]:
cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

In [10]:
r = requests.get('https://nominatim.openstreetmap.org/search?format=json&city=<Lille>')

In [11]:
r.content

b'[{"place_id":121668,"licence":"Data \xc2\xa9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"node","osm_id":26686577,"boundingbox":["50.4765654","50.7965654","2.9035282","3.2235282"],"lat":"50.6365654","lon":"3.0635282","display_name":"Lille, Nord, Hauts-de-France, France m\xc3\xa9tropolitaine, 59800, France","class":"place","type":"city","importance":0.7632037863087654,"icon":"https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png"},{"place_id":282028769,"licence":"Data \xc2\xa9 OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright","osm_type":"relation","osm_id":58404,"boundingbox":["50.6008264","50.6612596","2.9679677","3.125725"],"lat":"50.631133000000005","lon":"3.055031128074955","display_name":"Lille, Nord, Hauts-de-France, France m\xc3\xa9tropolitaine, France","class":"boundary","type":"administrative","importance":0.7632037863087654,"icon":"https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.

In [12]:
r.json()[0]

{'place_id': 121668,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 26686577,
 'boundingbox': ['50.4765654', '50.7965654', '2.9035282', '3.2235282'],
 'lat': '50.6365654',
 'lon': '3.0635282',
 'display_name': 'Lille, Nord, Hauts-de-France, France métropolitaine, 59800, France',
 'class': 'place',
 'type': 'city',
 'importance': 0.7632037863087654,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png'}

In [13]:
#We look to be able to request the API automatically with a loop, to avoid computing 35 cities manually
def singular_city_data(city):
    
    city_url = 'https://nominatim.openstreetmap.org/search?country=France&format=json&limit=1&city={}.'.format(city)
    r = requests.get(city_url)
    return r.json()[0]

In [14]:
singular_city_data('Lille')

{'place_id': 121668,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'node',
 'osm_id': 26686577,
 'boundingbox': ['50.4765654', '50.7965654', '2.9035282', '3.2235282'],
 'lat': '50.6365654',
 'lon': '3.0635282',
 'display_name': 'Lille, Nord, Hauts-de-France, France métropolitaine, 59800, France',
 'class': 'place',
 'type': 'city',
 'importance': 0.8732037863087654,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png'}

In [15]:
#We now need to loop over the same process requesting, and append 

clean_city_list = list()

def data_list_loop(cities_list):
    for city in cities_list:
        data_c = singular_city_data(city)
        city_data = {'place': city, **data_c}
        clean_city_list.append(city_data)
    return clean_city_list

In [16]:
data_cities = data_list_loop(cities)

In [17]:
type(data_cities)

list

In [18]:
len(data_cities)

33

### Convert our list to a dataframe

In [19]:
#Convert our list to a dataframe
city_data = pd.DataFrame(data_cities)
city_data.head()

,place,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon
0,Mont Saint Michel,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",place,islet,0.865437,NaN
1,St Malo,282098015,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534,"[48.5979853, 48.6949736, -2.0765246, -1.9367259]",48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",boundary,administrative,0.786467,https://nominatim.openstreetmap.org/ui/mapicon...
2,Bayeux,281962470,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",boundary,administrative,0.792700,https://nominatim.openstreetmap.org/ui/mapicon...
3,Le Havre,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",boundary,administrative,0.932333,https://nominatim.openstreetmap.org/ui/mapicon...
4,Rouen,122848,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",place,city,0.860073,https://nominatim.openstreetmap.org/ui/mapicon...


In [59]:
#Convert the dataframe to csv and store it away for now 

In [60]:
#city_data.to_csv('city_data.csv')

In [61]:
city_data = city_data[['place','lat','lon']]

### Temperature API call

In [25]:
#city_data['rainprecipitation'] = city_data.json()['daily'][0]
#Lat & lon parameters writing 
#Store our key in a separate file
from vapros import APIKEY

lat = 48.6359541
long = 1.511459954959514
APIKEY = APIKEY
data = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&exclude=current,minutely,hourly,alerts&lang=fr&appid={}".format(lat,long,APIKEY))
data.json()

{'lat': 48.636,
 'lon': 1.5115,
 'timezone': 'Europe/Paris',
 'timezone_offset': 3600,
 'daily': [{'dt': 1643544000,
   'sunrise': 1643527582,
   'sunset': 1643561275,
   'moonrise': 1643524380,
   'moonset': 1643552400,
   'moon_phase': 0.93,
   'temp': {'day': 8.96,
    'min': 3.62,
    'max': 9.43,
    'night': 3.62,
    'eve': 6.09,
    'morn': 4.26},
   'feels_like': {'day': 8.65, 'night': 0.62, 'eve': 4.9, 'morn': 1.9},
   'pressure': 1032,
   'humidity': 66,
   'dew_point': 2.96,
   'wind_speed': 4.19,
   'wind_deg': 275,
   'wind_gust': 9.3,
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'ciel dégagé',
     'icon': '01d'}],
   'clouds': 8,
   'pop': 0,
   'uvi': 1.18},
  {'dt': 1643630400,
   'sunrise': 1643613904,
   'sunset': 1643647773,
   'moonrise': 1643613960,
   'moonset': 1643643480,
   'moon_phase': 0.97,
   'temp': {'day': 7.25,
    'min': 1.73,
    'max': 7.62,
    'night': 1.73,
    'eve': 4.09,
    'morn': 5.37},
   'feels_like': {'day': 3.16,

In [26]:
#We look for main weather & snow volume to filter out cities pre-emptively
#I'm quite fond of sailing so I'll be looking primarily for average_windspeed over the next 7 days

for i in range(0,33):
    lat = city_data['lat'][i]
    long = city_data['lon'][i]
    r = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units=metric&exclude=current,minutely,hourly,alerts&lang=fr&appid={}".format(lat,long,APIKEY))
    city_data.loc[i,"felt_temperature_today"] = r.json()["daily"][0]["feels_like"]["day"]
    average_windspeed_over_7_days = (r.json()["daily"][1]['wind_speed'] + r.json()["daily"][2]['wind_speed'] + r.json()["daily"][3]['wind_speed'] + r.json()["daily"][4]['wind_speed'] + r.json()["daily"][5]['wind_speed'] + r.json()["daily"][6]['wind_speed'] + r.json()["daily"][7]['wind_speed'])/7
    city_data.loc[i,'Cloudiness'] = r.json()["daily"][0]['clouds']
    city_data.loc[i,'Average_windspeed_7days'] = average_windspeed_over_7_days

In [27]:
city_data

,place,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon,felt_temperature_today,Cloudiness,Average_windspeed_7days
0,Mont Saint Michel,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",place,islet,0.865437,NaN,9.66,52.0,6.851429
1,St Malo,282098015,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534,"[48.5979853, 48.6949736, -2.0765246, -1.9367259]",48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",boundary,administrative,0.786467,https://nominatim.openstreetmap.org/ui/mapicon...,8.56,83.0,7.990000
2,Bayeux,281962470,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",boundary,administrative,0.792700,https://nominatim.openstreetmap.org/ui/mapicon...,9.58,55.0,8.055714
3,Le Havre,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",boundary,administrative,0.932333,https://nominatim.openstreetmap.org/ui/mapicon...,8.94,7.0,9.432857
4,Rouen,122848,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",place,city,0.860073,https://nominatim.openstreetmap.org/ui/mapicon...,8.83,16.0,7.701429
5,Paris,281739181,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7444,"[48.8155755, 48.902156, 2.224122, 2.4697602]",48.8588897,2.3200410217200766,"Paris, Île-de-France, France métropolitaine, F...",boundary,administrative,1.051710,https://nominatim.openstreetmap.org/ui/mapicon...,9.02,15.0,5.344286
6,Amiens,120890,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686514,"[49.7341708, 50.0541708, 2.1356951, 2.4556951]",49.8941708,2.2956951,"Amiens, Somme, Hauts-de-France, France métropo...",place,city,0.844949,https://nominatim.openstreetmap.org/ui/mapicon...,8.25,13.0,8.447143
7,Lille,121668,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686577,"[50.4765654, 50.7965654, 2.9035282, 3.2235282]",50.6365654,3.0635282,"Lille, Nord, Hauts-de-France, France métropoli...",place,city,0.873204,https://nominatim.openstreetmap.org/ui/mapicon...,8.35,17.0,8.245714
8,Strasbourg,281833860,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,71033,"[48.491861, 48.6461896, 7.6881371, 7.8360646]",48.584614,7.7507127,"Strasbourg, Bas-Rhin, Grand Est, France métrop...",boundary,administrative,0.894805,https://nominatim.openstreetmap.org/ui/mapicon...,5.54,74.0,5.831429
9,Chateau du Haut Koenigsbourg,49750339,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,4245068168,"[48.2494726, 48.2495726, 7.3454423, 7.3455423]",48.2495226,7.3454923,"Château du Haut-Kœnigsbourg, Orschwiller, Séle...",place,isolated_dwelling,0.510000,NaN,2.54,55.0,4.078571


In [28]:
city_data["address"] = city_data["place"]

In [30]:
for i in range(0,33):
    city_data.loc[i,'unique_id'] = i

In [31]:
city_data = city_data.astype({"unique_id": int})
city_data.head()

,place,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,class,type,importance,icon,felt_temperature_today,Cloudiness,Average_windspeed_7days,address,unique_id
0,Mont Saint Michel,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",place,islet,0.865437,NaN,9.66,52.0,6.851429,Mont Saint Michel,0
1,St Malo,282098015,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,905534,"[48.5979853, 48.6949736, -2.0765246, -1.9367259]",48.649518,-2.0260409,"Saint-Malo, Ille-et-Vilaine, Bretagne, France ...",boundary,administrative,0.786467,https://nominatim.openstreetmap.org/ui/mapicon...,8.56,83.0,7.990000,St Malo,1
2,Bayeux,281962470,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,145776,"[49.2608124, 49.2934736, -0.7275671, -0.6757378]",49.2764624,-0.7024738,"Bayeux, Calvados, Normandie, France métropolit...",boundary,administrative,0.792700,https://nominatim.openstreetmap.org/ui/mapicon...,9.58,55.0,8.055714,Bayeux,2
3,Le Havre,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",boundary,administrative,0.932333,https://nominatim.openstreetmap.org/ui/mapicon...,8.94,7.0,9.432857,Le Havre,3
4,Rouen,122848,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,26686587,"[49.2804591, 49.6004591, 0.9339658, 1.2539658]",49.4404591,1.0939658,"Rouen, Seine-Maritime, Normandie, France métro...",place,city,0.860073,https://nominatim.openstreetmap.org/ui/mapicon...,8.83,16.0,7.701429,Rouen,4


In [35]:
city_data = city_data[['unique_id','place','lat','lon','felt_temperature_today','Cloudiness','Average_windspeed_7days']]
city_data["address"] = city_data["place"]
city_data = city_data[['unique_id','address','lat','lon','felt_temperature_today','Cloudiness','Average_windspeed_7days']]
city_data.head()

,unique_id,address,lat,lon,felt_temperature_today,Cloudiness,Average_windspeed_7days
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,9.66,52.0,6.851429
1,1,St Malo,48.649518,-2.0260409,8.56,83.0,7.990000
2,2,Bayeux,49.2764624,-0.7024738,9.58,55.0,8.055714
3,3,Le Havre,49.4938975,0.1079732,8.94,7.0,9.432857
4,4,Rouen,49.4404591,1.0939658,8.83,16.0,7.701429


## Convert the dataframe to csv 

In [99]:
#city_data.to_csv('city_weather_list.csv', index = False)

In [37]:
city_data.head()

,unique_id,address,lat,lon,felt_temperature_today,Cloudiness,Average_windspeed_7days
19,19,Marseille,43.2961743,5.3699525,11.48,0.0,12.730000
21,21,Avignon,43.9492493,4.8059012,10.75,1.0,12.071429
18,18,Cassis,43.2140359,5.5396318,11.85,0.0,11.767143
25,25,Saintes Maries de la mer,43.4522771,4.4287172,13.32,0.0,10.891429
26,26,Collioure,42.52505,3.0831554,17.26,1.0,10.110000


In [40]:
city_data = city_data.sort_values(by=['Average_windspeed_7days'], ascending=False)
city_data.head()

,unique_id,address,lat,lon,felt_temperature_today,Cloudiness,Average_windspeed_7days
19,19,Marseille,43.2961743,5.3699525,11.48,0.0,12.730000
21,21,Avignon,43.9492493,4.8059012,10.75,1.0,12.071429
18,18,Cassis,43.2140359,5.5396318,11.85,0.0,11.767143
25,25,Saintes Maries de la mer,43.4522771,4.4287172,13.32,0.0,10.891429
26,26,Collioure,42.52505,3.0831554,17.26,1.0,10.110000


In [41]:
#We can therefore compute a top 5 cities list for our criterion windspeed average

top5_cities_list = city_data.sort_values(by=['Average_windspeed_7days'], ascending=False)[:6]
top5_cities_list

,unique_id,address,lat,lon,felt_temperature_today,Cloudiness,Average_windspeed_7days
19,19,Marseille,43.2961743,5.3699525,11.48,0.0,12.730000
21,21,Avignon,43.9492493,4.8059012,10.75,1.0,12.071429
18,18,Cassis,43.2140359,5.5396318,11.85,0.0,11.767143
25,25,Saintes Maries de la mer,43.4522771,4.4287172,13.32,0.0,10.891429
26,26,Collioure,42.52505,3.0831554,17.26,1.0,10.110000
3,3,Le Havre,49.4938975,0.1079732,8.94,7.0,9.432857


In [42]:
#We're running into issues as the type of lat/long isn't float, we have to convert

top5_cities_list = top5_cities_list.astype({'lat': float,'lon': float,'Average_windspeed_7days': float, 'Cloudiness': float}) 

In [43]:
fig = px.scatter_mapbox(top5_cities_list, lat="lat", lon="lon", zoom = 4,
                        hover_data = ['Cloudiness', 'Average_windspeed_7days'], 
                        color_continuous_scale = 'sunset',
                        color="Average_windspeed_7days",
                        mapbox_style="carto-positron")

fig.show('iframe') 

## Exclude Avignon
As we're looking to go sailing, we'll have to exclude _Avignon_ the sea isn't near...

I'll get _Le Havre_ instead

## Second option 

I quite like when it's cloudy and grey as I'm used to this type of climate, I find it quite freshening, therefore I'll compute the locations where there is the most clouds 

In [48]:
top5_cities_list_cloud = city_data.sort_values(by=['Cloudiness'], ascending=False)[:5]
top5_cities_list_cloud = top5_cities_list_cloud.astype({'lat': float,'lon': float,'Average_windspeed_7days': float, 'Cloudiness': float}) 

In [50]:
top5_cities_list_cloud.head()

,unique_id,address,lat,lon,felt_temperature_today,Cloudiness,Average_windspeed_7days
32,32,La Rochelle,46.159113,-1.152043,8.06,100.0,5.051429
16,16,Lyon,45.757814,4.832011,6.90,96.0,3.237143
12,12,Besancon,47.238022,6.024362,6.02,96.0,4.557143
13,13,Dijon,47.321581,5.041470,6.67,92.0,5.097143
1,1,St Malo,48.649518,-2.026041,8.56,83.0,7.990000


In [49]:
fig = px.scatter_mapbox(top5_cities_list_cloud, lat="lat", lon="lon", zoom = 4,
                        hover_data = ['Cloudiness', 'Average_windspeed_7days'], 
                        color_continuous_scale = 'icefire',
                        color="Cloudiness",
                        mapbox_style="carto-positron")

fig.show('iframe') 

## Helpers 🦮

To help you achieve this project, here are a few tips that should help you

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

## Deliverable 📬

To complete this project, your team should deliver:

* A `.csv` file in an S3 bucket containing enriched information about weather and hotels for each french city

* A SQL Database where we should be able to get the same cleaned data from S3 

* Two maps where you should have a Top-5 destinations and a Top-20 hotels in the area. You can use plotly or any other library to do so. It should look something like this: 

![Map](https://full-stack-assets.s3.eu-west-3.amazonaws.com/images/Kayak_best_destination_project.png)